In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import tensorflow as tf
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow
import quantumflow.crazynet

In [ ]:
def input_function(x_inputs):
    return np.ones_like(x_inputs)

def target_function(x, x_inputs, inputs):
    return np.sum(np.sum(np.square(np.expand_dims(x, axis=-2) - x_inputs), axis=-1) < 1, axis=-1, keepdims=True)

In [ ]:
n_inputs = 5

x = np.expand_dims(np.linspace(0, 10, 1000), axis=-1)
x_inputs = np.random.randn(n_inputs, 1)+5
inputs = input_function(x_inputs)

targets = target_function(x, x_inputs, inputs)

print(x.shape)
print(x_inputs.shape)
print(inputs.shape)
print(targets.shape)

plt.figure(figsize=(20, 4))
plt.plot(x, targets)
plt.plot(x_inputs, inputs, '.k')
plt.show()

In [ ]:
num_outputs = 1
num_layers = 8
d_model = 128
dff_input = [64, 128]
dff = 128
xdff = [8, 16, 8]
dff_final = [128, 64]
num_heads = 1
dropout_rate = 0.0
scale = 1.0

batch_size = 256
steps = 10000

model = quantumflow.crazynet.CrazyNet(num_outputs=num_outputs, 
                                      num_layers=num_layers, 
                                      d_model=d_model, 
                                      num_heads=num_heads, 
                                      dff_input=dff_input, 
                                      dff=dff, 
                                      dff_final=dff_final, 
                                      dropout_rate=dropout_rate, 
                                      scale=scale)

def loss_fn(outputs, targets):
    return tf.reduce_mean(tf.square(outputs - targets))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


def gen_inputs(batch_size):
    x_inputs = (np.random.randn(batch_size, n_inputs, 1)+5).astype(np.float32)
    inputs = input_function(x_inputs)
    return x_inputs, inputs

def gen_x(x_inputs, inputs, batch_size):
    x = np.random.uniform(10, size=(batch_size, 1)).astype(np.float32)
    targets = target_function(x, x_inputs, inputs).astype(np.float32)
    return x, targets

@tf.function
def validate_call(x, x_inputs, inputs):
    return model(x, x_inputs, inputs)
    

def validate_fn(x_inputs=None, inputs=None):  
    if x_inputs is None:
        x_inputs, inputs = gen_inputs(1)     
    
    x_fine = np.expand_dims(np.linspace(0, 10, 1000).astype(np.float32), axis=-1)
    
    targets_fine = target_function(x_fine, x_inputs[0], inputs[0]).astype(np.float32)
    
    x_inputs = np.repeat(x_inputs, len(x_fine), axis=0)
    inputs = np.repeat(inputs, len(x_fine), axis=0)
    
    outputs_fine = validate_call(x_fine, x_inputs, inputs)
    
    return outputs_fine, targets_fine - outputs_fine

losses = []
validation = []
step = 0

In [ ]:
x_inputs, inputs = gen_inputs(1)
x, targets = gen_x(x_inputs[0], inputs[0], 1)

outputs = model(x, x_inputs, inputs)

plt.figure(figsize=(20, 3))
plt.plot(x_inputs[0], inputs[0], 'k.')
plt.plot(x[0], outputs[0], 'r.')
plt.plot(x[0], targets[0], 'g.')
plt.xlim([0, 10])
plt.ylim([-0.1, 1.1])
plt.show()

In [ ]:
@tf.function
def step_fn(x, targets, x_inputs, inputs):
    with tf.GradientTape() as tape:

        outputs = model(x, x_inputs, inputs, training=True) 

        loss_value = loss_fn(outputs, targets)

    grads = tape.gradient(loss_value, model.trainable_weights)
    
    #[print(var.name, np.sum(np.abs(grad))) for grad, var in zip(grads, model.trainable_weights)]

    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value


print(f"{'step':>10} {'loss':>20} {'mean(abs(err))':>20} {'max(abs(err))':>20}")

for _ in range(steps):
    x_inputs, inputs = gen_inputs(batch_size)
    x, targets = gen_x(x_inputs, inputs, batch_size)
    
    loss_value = step_fn(x, targets, x_inputs, inputs)
    
    losses.append(loss_value.numpy())
    if step % 100 == 0:
        output, err = validate_fn()
        print(f"{step:10d} {loss_value:20f} {np.mean(np.abs(err)):20f} {np.max(np.abs(err)):20f}")
        validation.append(output)
    
    if step > 2500:
        optimizer.learning_rate.assign(optimizer.learning_rate*0.999)
    
    step += 1

In [ ]:
plt.figure(figsize=(20, 3))
plt.plot(losses)
plt.yscale('log')
plt.show()

In [ ]:
x_inputs, inputs = gen_inputs(1)

outputs, err = validate_fn(x_inputs, inputs)
outputs_prev = outputs

plt.figure(figsize=(20, 3))
plt.plot(x_inputs[0], inputs[0], 'k.')
plt.plot(np.linspace(0, 10, 1000), outputs)
plt.plot(np.linspace(0, 10, 1000), err, 'r')
plt.xlim([0, 10])
plt.show()

In [ ]:
quantumflow.utils.anim_plot(validation, x=np.linspace(0, 10, 1000))

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')['peak']/1024**3
#0.08727812767028809